<a href="https://colab.research.google.com/github/lungnahahd/Tensorflow_Prac/blob/main/Reusing_Pretrained_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/11_training_deep_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml2/blob/master/11_training_deep_neural_networks.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

## Reusing Pretrained Layers
## Transfer Learning



### Reusing a Keras model

Let's split the fashion MNIST training set in two:
* `X_train_A`: all images of all items except for sandals and shirts (classes 5 and 6).
* `X_train_B`: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using `Dense` layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [5]:
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [8]:
X_train_A.shape

(43986, 28, 28)

In [9]:
X_train_B.shape

(200, 28, 28)

In [10]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [11]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

랜덤한 weight를 고정시켜주기 위해 작성한 코드 부분

In [ ]:

tf.random.set_seed(42)
np.random.seed(42)

In [13]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu")) ## selu를 통해 각 layer를 학습
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [14]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [15]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.5660 - accuracy: 0.8220 - val_loss: 0.3719 - val_accuracy: 0.8739
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3502 - accuracy: 0.8796 - val_loss: 0.3271 - val_accuracy: 0.8919
Epoch 3/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3163 - accuracy: 0.8910 - val_loss: 0.3014 - val_accuracy: 0.8994
Epoch 4/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2977 - accuracy: 0.8969 - val_loss: 0.2860 - val_accuracy: 0.9056
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2855 - accuracy: 0.9009 - val_loss: 0.2764 - val_accuracy: 0.9071
Epoch 6/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2763 - accuracy: 0.9050 - val_loss: 0.2723 - val_accuracy: 0.9096
Epoch 7/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2687 - accuracy: 0.9081 - val_loss: 0.2677 - val_accuracy:

In [16]:
model_A.save("my_model_A.h5")



> 여기서 부터 실질적인 Transfer Learning 시작

* **model A는 사전에 미리 학습 되었다고 가정**
* **model B는 transfer Learning을 모르는 사람이 비슷하지만 다른 주제를 가지고 learning을 시작하는 경우**



In [17]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

여기서는 model_A와 유사한 주제로 둘 중 하나를 선택하는 binary 문제를 해결하기 위한 모델을 만드는 것이라고 가정

In [18]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [19]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 44ms/step - loss: 0.8108 - accuracy: 0.4750 - val_loss: 0.6067 - val_accuracy: 0.7059
Epoch 2/20
7/7 [==============================] - 0s 15ms/step - loss: 0.5651 - accuracy: 0.7300 - val_loss: 0.4600 - val_accuracy: 0.8205
Epoch 3/20
7/7 [==============================] - 0s 17ms/step - loss: 0.4315 - accuracy: 0.8350 - val_loss: 0.3717 - val_accuracy: 0.8722
Epoch 4/20
7/7 [==============================] - 0s 15ms/step - loss: 0.3503 - accuracy: 0.8650 - val_loss: 0.3072 - val_accuracy: 0.9148
Epoch 5/20
7/7 [==============================] - 0s 17ms/step - loss: 0.2865 - accuracy: 0.9200 - val_loss: 0.2610 - val_accuracy: 0.9361
Epoch 6/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2410 - accuracy: 0.9450 - val_loss: 0.2293 - val_accuracy: 0.9422
Epoch 7/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2094 - accuracy: 0.9500 - val_loss: 0.2051 - val_accuracy: 0.9493
Epoch 8/20
7/7 [===========

In [20]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 300)               235500    
                                                                 
 dense_13 (Dense)            (None, 100)               30100     
                                                                 
 dense_14 (Dense)            (None, 50)                5050      
                                                                 
 dense_15 (Dense)            (None, 50)                2550      
                                                                 
 dense_16 (Dense)            (None, 50)                2550      
                                                                 
 dense_17 (Dense)            (None, 1)                

* **여기서부터는 model_A를 load해서 이를 활용한 model_B_on_A를 만드는 과정**

In [ ]:
model_A = keras.models.load_model("my_model_A.h5") # 기존 모델 가져오기
model_B_on_A = keras.models.Sequential(model_A.layers[:-1]) # 모델을 마지막 layer 전까지 가져온다는 의미
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # 가져온 모델 A에 사용자가 원하는 마지막 layer를 따로 추가해서 새로운 모델 생성

* 기존 모델을 그대로 위에처럼 사용하면, 학습 시 기존 모델 A에도 영향이 미친다.
* 이를 방지하기 위해서는 아래와 같이 clone을 통해 동일하지만 독립적인 모델을 따로 만들어서 진행해주면, 학습 시, 추가적인 모델도 만들 수 있고, 기존 모델도 온전히 보호하면서 작업을 진행할 수 있음
* 주의할 점은 모델을 clone하면 모델 자체만 복사되지 관련된 weight까지 가져오지는 않기에 wegiht를 따로 입혀주어야 함

In [22]:
model_A_clone = keras.models.clone_model(model_A) # 기존 모델을 clone 
model_A_clone.set_weights(model_A.get_weights()) # 모델을 clone하고 기존 모델의 weight 입히기
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1]) # 복사한 모델을 원하는 부분만 가져오기
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # 복사한 모델을 활용

In [24]:
# for문을 통해서 기존 학습된 모델의 부분은 false로 고정시켜놓기 가능
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
# 생성하기 원하는 모델의 compile 과정
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [25]:
# 생성하기 원하는 모델을 학습
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

# 학습을 완료한 이후에는 전부 이전에 false로 고정했던 것을 풀어주기
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
# 그리고 전체적인 모델을 활용해서 추가학습 진행
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 43ms/step - loss: 1.8778 - accuracy: 0.1550 - val_loss: 1.8435 - val_accuracy: 0.2373
Epoch 2/4
7/7 [==============================] - 0s 16ms/step - loss: 1.7316 - accuracy: 0.2100 - val_loss: 1.7059 - val_accuracy: 0.2606
Epoch 3/4
7/7 [==============================] - 0s 14ms/step - loss: 1.5926 - accuracy: 0.2400 - val_loss: 1.5774 - val_accuracy: 0.2819
Epoch 4/4
7/7 [==============================] - 0s 14ms/step - loss: 1.4635 - accuracy: 0.2650 - val_loss: 1.4546 - val_accuracy: 0.3114
Epoch 1/16
7/7 [==============================] - 1s 46ms/step - loss: 1.1468 - accuracy: 0.3650 - val_loss: 0.9000 - val_accuracy: 0.4817
Epoch 2/16
7/7 [==============================] - 0s 18ms/step - loss: 0.6921 - accuracy: 0.5600 - val_loss: 0.5966 - val_accuracy: 0.6673
Epoch 3/16
7/7 [==============================] - 0s 17ms/step - loss: 0.4492 - accuracy: 0.7650 - val_loss: 0.4188 - val_accuracy: 0.8012
Epoch 4/16
7/7 [===============

In [26]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0920 - accuracy: 0.9860


[0.09204290062189102, 0.9860000014305115]

transfer learning의 결과가 보다 완벽한 것을 확인 가능

In [27]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0843 - accuracy: 0.9940


[0.08425780385732651, 0.9940000176429749]

Great! We got quite a bit of transfer: the error rate dropped by a factor of 4.9!

In [ ]:
(100 - 97.05) / (100 - 99.40)

4.916666666666718